<a href="https://colab.research.google.com/github/akashqp/Neural-Machine-Translation/blob/main/Neural_Machine_Translation_by_Jointly_Learning_to_Align_and_Translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.6.0
!pip install sacrebleu
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.5 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 4.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

2023-03-02 08:40:52.769742: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 08:40:53.657875: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-02 08:40:53.657997: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-02 08:40:53.658017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator
from torchtext.data.metrics import bleu_score

import spacy

import random
import math
import time

In [ ]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG), root="data")

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 656kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 172kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 167kB/s]


In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim = 1))
        
        #output = [bsz, output dim]
        
        return output, hidden.squeeze(0)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, max_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,518,405 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX, reduction='none')

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss_per_sent = 0
    epoch_loss_per_tok = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        trg_sent_len = trg.shape[0]
                
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        #reshape to [trg sent len - 1, batch size]
        loss = loss.view(trg_sent_len-1, -1)
        
        with torch.no_grad():
            loss_per_tok = loss.mean()
        
        #sum loss across trg sent len dimension
        loss = loss.sum(0)
        
        #average across batch to get average loss per sentence
        loss = loss.mean(0)
                
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss_per_sent += loss.item()
        epoch_loss_per_tok += loss_per_tok.item()
        
    return epoch_loss_per_sent / len(iterator), epoch_loss_per_tok / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss_per_sent = 0
    epoch_loss_per_tok = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            trg_sent_len = trg.shape[0]
            
            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            #reshape to [trg sent len - 1, batch size]
            loss = loss.view(trg_sent_len - 1, -1)

            with torch.no_grad():
                loss_per_tok = loss.mean()
            
            #sum loss across trg sent len dimension
            loss = loss.sum(0)

            #average across batch to get average loss per sentence
            loss = loss.mean(0)
            
            epoch_loss_per_sent += loss.item()
            epoch_loss_per_tok += loss_per_tok.item()
        
    return epoch_loss_per_sent / len(iterator), epoch_loss_per_tok / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss_per_sent, train_loss_per_tok = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss_per_sent, valid_loss_per_tok = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss_per_sent < best_valid_loss:
        best_valid_loss = valid_loss_per_sent
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss Sen: {train_loss_per_sent:.3f} | Train PPL Sen: {math.exp(train_loss_per_sent):7.3f}')
    print(f'\t Val. Loss Sen: {valid_loss_per_sent:.3f} |  Val. PPL Sen: {math.exp(valid_loss_per_sent):7.3f}')
    print(f'\tTrain Loss Tok: {train_loss_per_tok:.3f} | Train PPL Tok: {math.exp(train_loss_per_tok):7.3f}')
    print(f'\t Val. Loss Tok: {valid_loss_per_tok:.3f} |  Val. PPL Tok: {math.exp(valid_loss_per_tok):7.3f}')

Epoch: 01 | Time: 1m 27s
	Train Loss Sen: 71.898 | Train PPL Sen: 16786041689371068429640069545984.000
	 Val. Loss Sen: 71.600 |  Val. PPL Sen: 12452930447689133929532481339392.000
	Train Loss Tok: 2.485 | Train PPL Tok:  11.996
	 Val. Loss Tok: 3.912 |  Val. PPL Tok:  50.023
Epoch: 02 | Time: 1m 27s
	Train Loss Sen: 62.452 | Train PPL Sen: 1326346400010269454197850112.000
	 Val. Loss Sen: 70.982 |  Val. PPL Sen: 6713407341127518575192247894016.000
	Train Loss Tok: 2.149 | Train PPL Tok:   8.572
	 Val. Loss Tok: 3.893 |  Val. PPL Tok:  49.061
Epoch: 03 | Time: 1m 27s
	Train Loss Sen: 55.036 | Train PPL Sen: 797620986649718714007552.000
	 Val. Loss Sen: 58.818 |  Val. PPL Sen: 35036373382994550841147392.000
	Train Loss Tok: 1.908 | Train PPL Tok:   6.739
	 Val. Loss Tok: 3.175 |  Val. PPL Tok:  23.926
Epoch: 04 | Time: 1m 28s
	Train Loss Sen: 44.004 | Train PPL Sen: 12901824837503956992.000
	 Val. Loss Sen: 52.679 |  Val. PPL Sen: 75546670536488740978688.000
	Train Loss Tok: 1.521 | Tra

In [ ]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss_per_sent, test_loss_per_tok = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss_per_sent:.3f} | Test PPL: {math.exp(test_loss_per_sent):7.3f} |')
print(f'| Test Loss: {test_loss_per_tok:.3f} | Test PPL: {math.exp(test_loss_per_tok):7.3f} |')

| Test Loss: 46.421 | Test PPL: 144721150610455461888.000 |
| Test Loss: 2.504 | Test PPL:  12.226 |


In [ ]:
nlp = spacy.load('en_core_web_sm')
def tokenize(text):
    return [tok.text for tok in nlp.tokenizer(text)]

In [ ]:
def translated_sentences(model, sentences, src_field, trg_field, device, max_len=50):
    # set the model to evaluation mode
    model.eval()

    # initialize the output list
    outputs = []

    # iterate through each sentence
    for sentence in sentences:
        # tokenize the source sentence and convert to tensor
        tokenized = tokenize(sentence)
        indexed = [src_field.vocab.stoi[t] for t in tokenized]
        src_tensor = torch.LongTensor(indexed).unsqueeze(1).to(device)

        # initialize the decoder hidden state
        with torch.no_grad():
            encoder_outputs, hidden = model.encoder(src_tensor)

        # initialize the input sequence as the start token
        trg_input = torch.LongTensor([trg_field.vocab.stoi[trg_field.init_token]]).to(device)

        # initialize the output list for this sentence
        output = []

        # decode the sequence step by step
        for i in range(max_len):
            # get the decoder output and hidden state for the current timestep
            with torch.no_grad():
                output_token, hidden = model.decoder(trg_input, hidden, encoder_outputs)

            # append the output token to the output list
            output.append(output_token.tolist()[0])

            # if the output token is the EOS token, stop decoding
            if trg_field.vocab.stoi[trg_field.eos_token] in output[-1]:
                break

            # prepare the input for the next timestep
            trg_input = output_token.argmax(dim=1).unsqueeze(0)

        # convert the output list to tokens and append to the outputs list
        tokens = [trg_field.vocab.itos[i] for i in output]
        outputs.append(tokens)

    # return the list of translated sentences
    return outputs

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu_score(model, data, src_field, trg_field, device, max_len=50):
    trgs = []
    pred_trgs = []

    for datum in data:
        # translate the sentence
        pred_tokens = translated_sentences(model, datum.src, src_field, trg_field, device, max_len)
        pred_trg = ' '.join(pred_tokens[:-1])  # remove the <eos> token
        pred_trgs.append(pred_trg)

        # remove punctuation from the target sentence
        trg = ' '.join(datum.trg).translate(str.maketrans('', '', string.punctuation))
        trgs.append(trg)

    # tokenize the target sentences
    trgs = [tokenize(trg) for trg in trgs]
    pred_trgs = [tokenize(pred_trg) for pred_trg in pred_trgs]

    # calculate the BLEU score
    return corpus_bleu(trgs, pred_trgs)

In [ ]:
# calculate BLEU score
bleu_score = calculate_bleu_score(model, test_data, SRC, TRG, device)

# print the BLEU score
print(f'BLEU score: {bleu_score:.4f}')

RuntimeError: ignored

In [ ]:
def calculate_bleu_score(model, iterator, trg_vocab):
    model.eval()
    
    predicted_translations = []
    target_translations = []

    with torch.no_grad():
        for batch in iterator:
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) # turn off teacher forcing

            # convert output from tensor to list of tokenized sentences
            output = [[trg_vocab.itos[token] for token in sentence] for sentence in output.argmax(dim=-1).transpose(0, 1)]

            # convert target from tensor to list of tokenized sentences
            trg = [[trg_vocab.itos[token] for token in sentence] for sentence in trg.transpose(0, 1)]

            predicted_translations.extend(output)
            target_translations.extend(trg)

    # remove the SOS and EOS tokens from the predicted translations
    predicted_translations = [[token for token in sentence if token not in ['<sos>', '<eos>']] for sentence in predicted_translations]

    # calculate BLEU score
    return bleu_score(predicted_translations, target_translations)

In [ ]:
bleu = calculate_bleu_score(model, test_iterator, TRG.vocab)
print(f'BLEU score: {bleu*100:.2f}')

BLEU score: 0.00


In [ ]:
def translate_sentence(model, sentence, src_field, trg_field, device, max_len=50):
    # Set the model to evaluation mode
    model.eval()

    # Tokenize the source sentence if it has not been tokenized
    if isinstance(sentence, str):
        nlp = spacy.load('de_core_news_sm')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # Add the start and end-of-sentence tokens
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]

    # Convert tokens to tensor on the source device
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # Pass the source tensor through the encoder
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)

    # Initialize the first input to the decoder
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    # Initialize variables to keep track of the length and the attention scores of the translation
    attentions = torch.zeros(max_len, 1, len(tokens)).to(device)

    # Translate sentence word by word
    for i in range(max_len):
        # Convert the target input to tensor on the target device
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        # Pass the target tensor and the previous hidden state and cell state (if any) through the decoder
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden, encoder_outputs)

        # Compute the attention scores and store them in the attentions tensor
        attention = model.attention(hidden, encoder_outputs)
        attentions[i] = attention.cpu().data

        # Get the index of the predicted next word
        pred_token = output.argmax(1).item()

        # Add the predicted next word index to the list of indices
        trg_indexes.append(pred_token)

        # Stop translating if the end-of-sentence token was predicted
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # Convert the list of indices to the list of tokens
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # Remove the start-of-sentence token from the list of tokens
    trg_tokens = trg_tokens[1:]

    # Remove the padding tokens from the list of tokens
    trg_tokens = [token for token in trg_tokens if token != trg_field.pad_token]

    # Convert the attention scores to the list of attention matrices
    attention_matrices = [matrix.squeeze(1).cpu().numpy() for matrix in attentions[:i+1]]

    # Return the list of tokens and the list of attention matrices
    return trg_tokens, attention_matrices

AttributeError: ignored

In [ ]:
def calculate_bleu(model, iterator, max_len = 50):
    trgs = []
    pred_trgs = []
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            #output = [(trg sent len - 1) * batch size, output dim]
            #trg = [(trg sent len - 1) * batch size]

            pred_trg = output.argmax(1)

            #pred_trg = [(trg sent len - 1) * batch size]

            pred_trg = pred_trg.view(-1, batch_size).cpu().numpy().tolist()
            trg = trg.view(-1, batch_size).cpu().numpy().tolist()

            pred_trg = [[TRG.vocab.itos[i] for i in row] for row in pred_trg]
            trg = [[TRG.vocab.itos[i] for i in row] for row in trg]

            pred_trgs.extend(pred_trg)
            trgs.extend(trg)

    return bleu_score(pred_trgs, trgs, max_n = 4, weights = [0.25, 0.25, 0.25, 0.25])

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    # Set the model to evaluation mode
    model.eval()

    # Convert tensor to list of integers
    if isinstance(sentence, torch.Tensor):
        sentence = sentence.tolist()

    # Tokenize the source sentence if it has not been tokenized
    if isinstance(sentence, str):
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [src_field.vocab.itos[token] for token in sentence]

    # Add start and end of sentence tokens
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]

    # Convert tokens to tensor format
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    # Generate translations
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)

        trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
        for i in range(max_len):
            trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

            with torch.no_grad():
                output, hidden = model.decoder(trg_tensor, hidden, encoder_outputs)

            pred_token = output.argmax(1).item()
            trg_indexes.append(pred_token)

            if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
                break

    # Convert indices to tokens
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # Remove start token
    trg_tokens = trg_tokens[1:]

    # Remove end token
    if trg_field.eos_token in trg_tokens:
        trg_tokens = trg_tokens[:trg_tokens.index(trg_field.eos_token)]

    # Join tokens to form sentence
    translation = ' '.join(trg_tokens)
    return translation

In [ ]:
import sacrebleu

def calculate_bleu_score(dataset, src_field, trg_field, model, device):
    translations = []
    targets = []
    for example in dataset:
        src = vars(example)['src']
        trg = vars(example)['trg']
        translation, _ = translate_sentence(src, src_field, trg_field, model, device)
        translation = translation[:-1]
        translations.append(translation)
        targets.append([trg_field.vocab.itos[i] for i in trg][1:-1])
    return sacrebleu.corpus_bleu(translations, [targets])

In [ ]:
from rouge import Rouge

def calculate_rouge_score(dataset, src_field, trg_field, model, device):
    rouge = Rouge()
    scores = {'rouge-1': {'f': 0, 'p': 0, 'r': 0}, 'rouge-2': {'f': 0, 'p': 0, 'r': 0}}
    for example in dataset:
        src = vars(example)['src']
        trg = vars(example)['trg']
        translation, _ = translate_sentence(src, src_field, trg_field, model, device)
        translation = ' '.join(translation[:-1])
        target = ' '.join([trg_field.vocab.itos[i] for i in trg][1:-1])
        score = rouge.get_scores(translation, target)
        scores['rouge-1']['f'] += score[0]['rouge-1']['f']
        scores['rouge-1']['p'] += score[0]['rouge-1']['p']
        scores['rouge-1']['r'] += score[0]['rouge-1']['r']
        scores['rouge-2']['f'] += score[0]['rouge-2']['f']
        scores['rouge-2']['p'] += score[0]['rouge-2']['p']
        scores['rouge-2']['r'] += score[0]['rouge-2']['r']
        size = len(dataset)
        scores['rouge-1']['f'] /= size
        scores['rouge-1']['p'] /= size
        scores['rouge-1']['r'] /= size
        scores['rouge-2']['f'] /= size
        scores['rouge-2']['p'] /= size
        scores['rouge-2']['r'] /= size
        return scores

In [ ]:
# Calculate BLEU score
bleu_score = calculate_bleu_score(test_iterator, SRC, TRG, model, device)
print(f'BLEU score: {bleu_score.score:.4f}')

# Calculate ROUGE score
rouge_scores = calculate_rouge_score(test_iterator, SRC, TRG, model, device)
print(f'ROUGE-1: F={rouge_scores["rouge-1"]["f"]:.4f}, P={rouge_scores["rouge-1"]["p"]:.4f}, R={rouge_scores["rouge-1"]["r"]:.4f}')
print(f'ROUGE-2: F={rouge_scores["rouge-2"]["f"]:.4f}, P={rouge_scores["rouge-2"]["p"]:.4f}, R={rouge_scores["rouge-2"]["r"]:.4f}')

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(model, iterator, max_len = 50):
    trgs = []
    pred_trgs = []
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            #output = [(trg sent len - 1) * batch size, output dim]
            #trg = [(trg sent len - 1) * batch size]

            pred_trg = output.argmax(1)

            #pred_trg = [(trg sent len - 1) * batch size]

            pred_trg = pred_trg.view(-1, batch_size).cpu().numpy().tolist()
            trg = trg.view(-1, batch_size).cpu().numpy().tolist()

            pred_trg = [[TRG.vocab.itos[i] for i in row] for row in pred_trg]
            trg = [[TRG.vocab.itos[i] for i in row] for row in trg]

            pred_trgs.extend(pred_trg)
            trgs.extend(trg)

    return bleu_score(pred_trgs, trgs, max_n = 4, weights = [0.25, 0.25, 0.25, 0.25])